## TUTORIAL 06 - Unsteady Thermal block problem
**_Keywords: POD-Galerkin method, scalar problem_**

### 1. Introduction
In this Tutorial, we consider unsteady heat conduction in a two-dimensional domain $\Omega$.

<img src="data/thermal_block.png" />

We define two subdomains $\Omega_1$ and $\Omega_2$, such that
1. $\Omega_1$ is a disk centered at the origin of radius $r_0=0.5$, and
2. $\Omega_2=\Omega/\ \overline{\Omega_1}$. 

The conductivity $\kappa$ is assumed to be constant on $\Omega_1$ and $\Omega_2$, i.e.
$$
\kappa|_{\Omega_1}=\kappa_0 \quad \textrm{and} \quad \kappa|_{\Omega_2}=1.
$$

For this problem, we consider $P=2$ parameters:
1. the first one is related to the conductivity in $\Omega_1$, i.e. $\mu_0\equiv\kappa_0$ (_note that parameters numbering is zero-based_);
2. the second parameter $\mu_1$ takes into account the constant heat flux over $\Gamma_{base}$.

The parameter vector $\boldsymbol{\mu}$ is thus given by 
$$
\boldsymbol{\mu} = (\mu_0,\mu_1)
$$
on the parameter domain
$$
\mathbb{P}=[0.1,10]\times[-1,1].
$$

In this problem we model the heat transfer process due to the heat flux over the bottom boundary $\Gamma_{base}$ and the following conditions on the remaining boundaries:
* the left and right boundaries $\Gamma_{side}$ are insulated,
* the top boundary $\Gamma_{top}$ is kept at a reference temperature (say, zero),

with the aim of measuring the average temperature on $\Gamma_{base}$.

In order to obtain a faster approximation of the problem we pursue a model reduction by means of a POD-Galerkin reduced order method.

### 2. Parametrized formulation

Let $u(t;\boldsymbol{\mu})$ be the temperature in the domain $\Omega\times[0,t_f]$.

The strong formulation of the parametrized problem is given by:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, for $t\in[0,t_f]$, find $u(t;\boldsymbol{\mu})$ such that</center>

$$
\begin{cases}
	\partial_tu(t;\boldsymbol{\mu})- \text{div} (\kappa(\mu_0)\nabla u(t;\boldsymbol{\mu})) = 0 & \text{in } \Omega\times[0,t_f],\\
    u(t=0;\boldsymbol{\mu}) = 0 & \text{in } \Omega, \\ 
	u(t;\boldsymbol{\mu}) = 0 & \text{on } \Gamma_{top}\times[0,t_f],\\
	\kappa(\mu_0)\nabla u(t;\boldsymbol{\mu})\cdot \mathbf{n} = 0 & \text{on } \Gamma_{side}\times[0,t_f],\\
	\kappa(\mu_0)\nabla u(t;\boldsymbol{\mu})\cdot \mathbf{n} = \mu_1 & \text{on } \Gamma_{base}\times[0,t_f].
\end{cases}
$$
<br>

where 
* $\mathbf{n}$ denotes the outer normal to the boundaries $\Gamma_{side}$ and $\Gamma_{base}$,
* the conductivity $\kappa(\mu_0)$ is defined as follows:
$$
\kappa(\mu_0) =
\begin{cases}
	\mu_0 & \text{in } \Omega_1,\\
	1 & \text{in } \Omega_2,\\
\end{cases}
$$

The corresponding weak formulation reads:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, for $t\in[0,t_f]$, find $u(t;\boldsymbol{\mu})\in\mathbb{V}$ such that</center>

$$m\left(\partial_tu(t;\boldsymbol{\mu}),v;\boldsymbol{\mu}\right) + a\left(u(t;\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V},\quad \forall t\in[0,t_f]$$

where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in H^1(\Omega) : v|_{\Gamma_{top}}=0\}
$$
* the parametrized bilinear form $m(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$m(u, v;\boldsymbol{\mu})=\int_{\Omega} \partial_tu(t)v \ d\boldsymbol{x},$$
* the parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(u, v;\boldsymbol{\mu})=\int_{\Omega} \kappa(\mu_0)\nabla u\cdot \nabla v \ d\boldsymbol{x},$$
* the parametrized linear form $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(v; \boldsymbol{\mu})= \mu_1\int_{\Gamma_{base}}v \ ds,$$

The (compliant) output of interest $s(t;\boldsymbol{\mu})$ is given by
$$s(t;\boldsymbol{\mu}) = \mu_1\int_{\Gamma_{base}} u(t;\boldsymbol{\mu})$$
is computed for each $\boldsymbol{\mu}$.

In [2]:
import sys
sys.path.append("../../")

from mlnics import NN, Losses, Normalization, RONNData
from dolfin import *
from rbnics import *
import torch
import numpy as np

torch.manual_seed(0)
np.random.seed(0)

## 3. Affine decomposition

For this problem the affine decomposition is straightforward:
$$m(u,v;\boldsymbol{\mu})=\underbrace{1}_{\Theta^{m}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega}uv \ d\boldsymbol{x}}_{m_0(u,v)},$$
$$a(u,v;\boldsymbol{\mu})=\underbrace{\mu_0}_{\Theta^{a}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega_1}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_0(u,v)} \ + \  \underbrace{1}_{\Theta^{a}_1(\boldsymbol{\mu})}\underbrace{\int_{\Omega_2}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_1(u,v)},$$
$$f(v; \boldsymbol{\mu}) = \underbrace{\mu_1}_{\Theta^{f}_0(\boldsymbol{\mu})} \underbrace{\int_{\Gamma_{base}}v \ ds}_{f_0(v)}.$$
We will implement the numerical discretization of the problem in the class
```
class UnsteadyThermalBlock(ParabolicCoerciveProblem):
```
by specifying the coefficients $\Theta^{m}_*(\boldsymbol{\mu})$, $\Theta^{a}_*(\boldsymbol{\mu})$ and $\Theta^{f}_*(\boldsymbol{\mu})$ in the method
```
    def compute_theta(self, term):     
```
and the bilinear forms $m_*(u, v)$, $a_*(u, v)$ and linear forms $f_*(v)$ in
```
    def assemble_operator(self, term):
```

In [3]:
class UnsteadyThermalBlock(ParabolicCoerciveProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        ParabolicCoerciveProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)

    # Return custom problem name
    def name(self):
        return "UnsteadyThermalBlock1POD"

    # Return theta multiplicative terms of the affine expansion of the problem.
    def compute_theta(self, term):
        mu = self.mu
        if term == "m":
            theta_m0 = 1.
            return (theta_m0, )
        elif term == "a":
            theta_a0 = mu[0]
            theta_a1 = 1.
            return (theta_a0, theta_a1)
        elif term == "f":
            theta_f0 = mu[1]
            return (theta_f0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "m":
            u = self.u
            m0 = u * v * dx
            return (m0, )
        elif term == "a":
            u = self.u
            a0 = inner(grad(u), grad(v)) * dx(1)
            a1 = inner(grad(u), grad(v)) * dx(2)
            return (a0, a1)
        elif term == "f":
            ds = self.ds
            f0 = v * ds(1)
            return (f0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 3)]
            return (bc0,)
        elif term == "inner_product":
            u = self.u
            x0 = inner(grad(u), grad(v)) * dx
            return (x0,)
        elif term == "projection_inner_product":
            u = self.u
            x0 = u * v * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main program
### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [4]:
mesh = Mesh("data/thermal_block.xml")
subdomains = MeshFunction("size_t", mesh, "data/thermal_block_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/thermal_block_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1, two components)

In [5]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the UnsteadyThermalBlock class

In [6]:
problem = UnsteadyThermalBlock(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.1, 10.0), (-1.0, 1.0)]
problem.set_mu_range(mu_range)
problem.set_time_step_size(0.05)
problem.set_final_time(3)

### 4.4. Prepare reduction with a POD-Galerkin method

In [7]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(20, nested_POD=4)
reduction_method.set_tolerance(1e-8, nested_POD=1e-4)

### 4.5. Perform the offline phase

In [12]:
reduction_method.initialize_training_set(5)
reduced_problem = reduction_method.offline()

=          UnsteadyThermalBlock1POD POD-Galerkin offline phase begins          =

###################################### 0 #######################################
truth solve for mu = (5.533253688880515, 0.43037873274483895)
# t = 0.05
# t = 0.1
# t = 0.15
# t = 0.2
# t = 0.25
# t = 0.3
# t = 0.35
# t = 0.4
# t = 0.45
# t = 0.5
# t = 0.55
# t = 0.6
# t = 0.65
# t = 0.7
# t = 0.75
# t = 0.8
# t = 0.85
# t = 0.9
# t = 0.95
# t = 1
# t = 1.05
# t = 1.1
# t = 1.15
# t = 1.2
# t = 1.25
# t = 1.3
# t = 1.35
# t = 1.4
# t = 1.45
# t = 1.5
# t = 1.55
# t = 1.6
# t = 1.65
# t = 1.7
# t = 1.75
# t = 1.8
# t = 1.85
# t = 1.9
# t = 1.95
# t = 2
# t = 2.05
# t = 2.1
# t = 2.15
# t = 2.2
# t = 2.25
# t = 2.3
# t = 2.35
# t = 2.4
# t = 2.45
# t = 2.5
# t = 2.55
# t = 2.6
# t = 2.65
# t = 2.7
# t = 2.75
# t = 2.8
# t = 2.85
# t = 2.9
# t = 2.95
# t = 3
# t = 3.05
Total time steps 61 (0 rejected, 0 SNES fails)
update snapshots matrix
lambda_0 = 17.438977128361717
lambda_1 = 0.47617492874430406
lambda_2

In [27]:
dir(reduced_problem), dir(reduction_method)

(['AlreadyAppliedBaseDecorators',
  'N',
  'N_bc',
  'OperatorExpansionStorage',
  'ProblemSolver',
  'Q',
  'Q_ic',
  'T',
  '__abstractmethods__',
  '__class__',
  '__delattr__',
  '__dict__',
  '__dir__',
  '__doc__',
  '__eq__',
  '__format__',
  '__ge__',
  '__getattribute__',
  '__gt__',
  '__hash__',
  '__init__',
  '__init_subclass__',
  '__le__',
  '__lt__',
  '__module__',
  '__ne__',
  '__new__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__setattr__',
  '__sizeof__',
  '__str__',
  '__subclasshook__',
  '__weakref__',
  '_abc_impl',
  '_build_reduced_initial_condition',
  '_build_reduced_inner_products',
  '_build_reduced_operators',
  '_cache_key_from_N_and_kwargs',
  '_combine_all_inner_products',
  '_combine_all_projection_inner_products',
  '_combine_and_homogenize_all_dirichlet_bcs',
  '_combined_and_homogenized_dirichlet_bc',
  '_combined_inner_product',
  '_combined_projection_inner_product',
  '_compute_error',
  '_compute_error_output',
  '_compute_output'

In [32]:
reduced_problem.operator

{'a': <rbnics.backends.online.numpy.affine_expansion_storage.AffineExpansionStorage at 0x135aad090>,
 'f': <rbnics.backends.online.numpy.affine_expansion_storage.AffineExpansionStorage at 0x135aafd60>,
 's': <rbnics.backends.online.numpy.affine_expansion_storage.AffineExpansionStorage at 0x135aafc10>,
 'm': <rbnics.backends.online.numpy.affine_expansion_storage.AffineExpansionStorage at 0x135aafb80>}

In [30]:
reduced_problem.set_mu((1.0, 1.0))
reduced_problem._build_reduced_initial_condition()
reduced_problem.ProblemSolver.ic_eval(reduction_method)

AttributeError: 'ParabolicPODGalerkinReduction' object has no attribute 'problem'

In [ ]:
net = NN.RONN(problem, reduction_method)

In [ ]:
input_normalization = Normalization.StandardNormalization()
output_normalization = Normalization.IdentityNormalization()

In [ ]:
pdnn_loss = Losses.PDNN_Loss(net, output_normalization)
pinn_loss = Losses.PINN_Loss(net)
prnn_loss = Losses.PRNN_Loss(net, output_normalization, omega=1.)

In [ ]:
data = RONNData.RONNDataLoader(net)
_ = data.train_validation_split(0.2)

In [ ]:
_ = NN.normalize_and_train(net, data, pdnn_loss, input_normalization, lr=0.0001, epochs=100000, use_validation=True)

### 4.6. Perform an online solve

In [ ]:
online_mu = (8.0, -1.0)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()
plot(reduced_solution, reduced_problem=reduced_problem, every=5, interval=500)

In [ ]:
net.eval()

with torch.no_grad():
    online_mu_nn = torch.tensor(online_mu)
    # NOTE: Change output_normalization accordingly for the different losses
    reduced_solution_nn = net.solve(online_mu_nn, input_normalization, output_normalization=None)

net.train()
plot(reduced_solution_nn, reduced_problem=reduced_problem, every=5, interval=500)

In [ ]:
test = NN.get_test(net)
NN.error_analysis_fixed_net(net, test, input_normalization)

### 4.7. Perform an error analysis

In [ ]:
reduction_method.initialize_testing_set(10)
reduction_method.error_analysis()

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.initialize_testing_set(10)
reduction_method.speedup_analysis()